In [1]:
import pandas as pd
import pickle
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
#look at the "exploded" df with sponsoring units, and 

In [3]:
sponsUnitResearchers = pd.read_pickle("Files/Dataframes/researchers_sponsoringUnits.pkl")

In [4]:
#data for 2010 or earlier
startData = sponsUnitResearchers[sponsUnitResearchers.Year <= 2010]

In [12]:
#totalSize = startData.groupby('SponsoringUnitName').count()['SearchName'].reset_index()
sizePerYear = startData.groupby(['SponsoringUnitName', 'Year']).count()['SearchName'].reset_index().rename(columns = {'SearchName': 'UnitSize'})

In [21]:
invited = startData[startData.SessionType == 'invited']
attendees = startData[startData.SessionType == 'attendee']

In [22]:
invitedPerYear = invited.groupby(['SponsoringUnitName', 'Year']).count()['SearchName'].reset_index().rename(columns = {'SearchName': 'n_invited'})
attendeesPerYear = attendees.groupby(['SponsoringUnitName', 'Year']).count()['SearchName'].reset_index().rename(columns = {'SearchName': 'n_attendees'})

In [183]:
#find female ratio for each sponsoring unit
sponsoringUnits = startData.SponsoringUnitName.unique()
index = 0
femaleRatioInv_dict = {}
femaleRatioAtt_dict = {}
for i in sponsoringUnits:
    if i != None:
        sponsoringUnitInv = invited[invited.SponsoringUnitName == i]
        sponsoringUnitAtt = attendees[attendees.SponsoringUnitName == i]
        invited_crossTab = pd.crosstab(index = sponsoringUnitInv.Year, columns = sponsoringUnitInv.Gender,
                                      normalize = "index")['Female'].reset_index().rename(columns = {'Female': 'FemaleRatio_inv'})
        invited_crossTab['SponsoringUnitName'] = i
        if len(sponsoringUnitAtt) > 0: #if any attendees in sponsoring unit
            attendees_crossTab = pd.crosstab(index = sponsoringUnitAtt.Year, columns = sponsoringUnitAtt.Gender,
                                        normalize = 'index')['Female'].reset_index().rename(columns = {'Female': 'FemaleRatio_att'})
            attendees_crossTab['SponsoringUnitName'] = i
        
        invitedDict = invited_crossTab.to_dict('list')
        attendeesDict = attendees_crossTab.to_dict('list')
        femaleRatioInv_dict[index] = invitedDict
        femaleRatioAtt_dict[index] = attendeesDict
        index += 1

In [185]:
femaleRatio_inv = pd.DataFrame.from_dict(femaleRatioInv_dict, orient='index').explode(['Year', 'FemaleRatio_inv', 'SponsoringUnitName']).reset_index(drop = True)
femaleRatio_att = pd.DataFrame.from_dict(femaleRatioAtt_dict, orient='index').explode(['Year', 'FemaleRatio_att', 'SponsoringUnitName']).reset_index(drop = True) 

In [188]:
sponsorUnit_df = sizePerYear.merge(invitedPerYear, how = 'outer', on = ['SponsoringUnitName', 'Year'])
sponsorUnit_df = sponsorUnit_df.merge(attendeesPerYear, how = 'outer', on = ['SponsoringUnitName', 'Year'])
sponsorUnit_df = sponsorUnit_df.merge(femaleRatio_inv, how = 'outer', on = ['SponsoringUnitName', 'Year'])
sponsorUnit_df = sponsorUnit_df.merge(femaleRatio_att, how = 'outer', on = ['SponsoringUnitName', 'Year'])


sponsorUnit_df

,SponsoringUnitName,Year,UnitSize,n_invited,n_attendees,FemaleRatio_inv,FemaleRatio_att
0,"Atomic, Molecular & Optical Physics",2005,159,20,139.0,0.1,0.093525
1,"Atomic, Molecular & Optical Physics",2006,175,14,161.0,0.071429,0.111801
2,"Atomic, Molecular & Optical Physics",2007,357,15,342.0,0.266667,0.160819
3,"Atomic, Molecular & Optical Physics",2008,280,16,264.0,0.0625,0.170455
4,"Atomic, Molecular & Optical Physics",2009,265,28,237.0,0.214286,0.139241
...,...,...,...,...,...,...,...
95,Statistical and Nonlinear Physics,2006,357,30,327.0,0.1,0.17737
96,Statistical and Nonlinear Physics,2007,441,43,398.0,0.139535,0.188442
97,Statistical and Nonlinear Physics,2008,415,32,383.0,0.15625,0.154047
98,Statistical and Nonlinear Physics,2009,411,34,377.0,0.205882,0.153846


In [175]:
#standardize, and have each year as column as well
#so we only have one row per sponsoring unit
#it would be like this:
#SponsoringUnitName, UnitSizeY2005, UnitSizeY2005, n_invitedY2005


In [ ]:
#merge femaleRatio_inv and femaleRatio_Att 
#do outer joi

In [25]:
sponsoringUnits

,SponsoringUnitName,Year,UnitSize,n_invited,n_attendees
0,"Atomic, Molecular & Optical Physics",2005,159,20,139.0
1,"Atomic, Molecular & Optical Physics",2006,175,14,161.0
2,"Atomic, Molecular & Optical Physics",2007,357,15,342.0
3,"Atomic, Molecular & Optical Physics",2008,280,16,264.0
4,"Atomic, Molecular & Optical Physics",2009,265,28,237.0
...,...,...,...,...,...
89,Statistical and Nonlinear Physics,2006,357,30,327.0
90,Statistical and Nonlinear Physics,2007,441,43,398.0
91,Statistical and Nonlinear Physics,2008,415,32,383.0
92,Statistical and Nonlinear Physics,2009,411,34,377.0


In [ ]:
#find null values

In [102]:
#calculate gender ratio
genderRatio_invited = pd.crosstab(index=invited.Year,
                             columns=invited.Gender,
                             normalize="index").rename(columns = {'Female': 'Female invited', 'Male': 'Male invited'})

genderRatio_attendees = pd.crosstab(index=attendees.Year,
                             columns=attendees.Gender,
                             normalize="index").rename(columns = {'Female': 'Female attendees', 'Male': 'Male attendees'})

In [94]:
#we need to change the genderRatio to do it per Year AND sponsoring unit.
#Maybe another method is better


#femaleInv_ratio = genderRatio_invited['Female invited']
#femaleAtt_ratio = genderRatio_attendees['Female attendees']

#femaleInvAtt_ratio = femaleInv_ratio/femaleAtt_ratio

In [ ]:
#standardize, and weight it toward gender equality having the biggest 

In [103]:
femaleInv_ratio
#merge
#totalSize.reset_index()

Year
2005    0.152661
2006    0.145923
2007    0.124654
2008    0.149315
2009    0.144872
2010    0.167482
Name: Female invited, dtype: float64

In [ ]:
use K = 1 for KNN, so we only find the most similar

In [63]:
startData

,SearchName,FirstName,ResearcherCount,MaxPaperCount,API_Field,API_FieldCount,ExternalField,ModelField,ExternalFieldCount,ModelFieldCount,...,Title,SponsoringUnit,SessionIndex,SessionType,Seniority,CumulativePaperCount,Productivity,Gender,SponsoringUnits_all,SponsoringUnitName
3,holger waalkens,holger,5,4,Physics,3,Physics,Physics,3,4,...,Quantum Transition State Theory,DCP,127987.0,invited,2,2,1.000000,Male,DCP,Chemical Physics
10,john boone,john,44,6,Medicine,5,Medicine,Medicine,5,6,...,Dedicated breast CT,FPS DBP,165259.0,invited,16,4,0.250000,Male,FPS,Physics and Society
11,john boone,john,44,6,Medicine,5,Medicine,Medicine,5,6,...,Dedicated breast CT,FPS DBP,165259.0,invited,16,4,0.250000,Male,DBP,None
18,teemu laurila,teemu,1,16,Materials Science,7,Materials Science,Physics,7,13,...,Boiling Dynamics with Diffuse Interface Methods,APS,127353.0,attendee,6,8,1.333333,Female,APS,None
19,juha ruokolainen,juha,1,32,Computer Science,9,Computer Science,Engineering,9,21,...,Boiling Dynamics with Diffuse Interface Methods,APS,127353.0,attendee,17,12,0.705882,Male,APS,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641443,michelle mok,michelle,10,3,Political Science,1,Political Science,Computer Science,1,2,...,Drop shape analysis of poly-styrene acrylic ac...,None,104510.0,attendee,7,3,0.428571,Female,None,None
641444,michelle mok,michelle,10,3,Political Science,1,Political Science,Computer Science,1,2,...,Effects of Gradient Sequencing in Copolymers f...,DPOLY,136098.0,attendee,8,3,0.375000,Female,DPOLY,Polymer Physics
641445,hajime shibata,hajime,20,37,Computer Science,29,Computer Science,Engineering,29,37,...,Figure of merit for thermoelectric power gener...,FIAP DMP,53674.0,attendee,6,11,1.833333,Male,FIAP,Industrial and Applied Physics
641446,hajime shibata,hajime,20,37,Computer Science,29,Computer Science,Engineering,29,37,...,Figure of merit for thermoelectric power gener...,FIAP DMP,53674.0,attendee,6,11,1.833333,Male,DMP,Materials Physics


In [57]:
sponsUnitResearchers
#sponsUnitResearchers.groupby(['SponsoringUnitName', 'SessionType']).count()

#use femaleRatio for invited speakers as well

SearchName  FirstName  \
SponsoringUnitName                  SessionType                          
Atomic, Molecular & Optical Physics attendee          11871      11871   
                                    invited             564        564   
Biological Physics                  attendee          22096      22096   
                                    invited            1209       1209   
Chemical Physics                    attendee          11505      11505   
                                    invited             973        973   
Computational Physics               attendee          30552      30552   
                                    invited            1436       1436   
Condensed Matter Physics            attendee          63227      63227   
                                    invited            2944       2944   
Data Science                        attendee           2452       2452   
                                    invited             115        115   
Diversity and Inclusion             attendee              2          2   
                                    invited               7          7   
Early Career Scientists             attendee              7          7   
                                    invited              61         61   
Education                           attendee            617        617   
                                    invited             218        218   
Energy Research and Applications    attendee           5963       5963   
                                    invited             197        197   
Fluid Dynamics                      attendee          10705      10705   
                                    invited             249        249   
Graduate Student Affairs            attendee             15         15   
                                    invited              75         75   
History and Philosophy of Physics   invited               8          8   
Industrial and Applied Physics      attendee          22461      22461   
                                    invited            1095       1095   
Instrument and Measurement Science  attendee           5148       5148   
                                    invited             231        231   
International Physics               attendee             22         22   
                                    invited             184        184   
Laser Science                       attendee            683        683   
                                    invited              56         56   
Magnetism and its Applications      attendee          36114      36114   
                                    invited            1283       1283   
Materials Physics                   attendee          87361      87361   
                                    invited            2729       2729   
Medical Physics                     attendee            738        738   
                                    invited              61         61   
Nuclear Physics                     invited               3          3   
Outreach and Engaging the Public    attendee            213        213   
                                    invited              49         49   
Physics and Society                 attendee             14         14   
                                    invited             203        203   
Physics of Beams                    attendee             20         20   
                                    invited              24         24   
Physics of Climate                  attendee            377        377   
                                    invited              65         65   
Polymer Physics                     attendee          30633      30633   
                                    invited            1538       1538   
Quantum Information                 attendee          17067      17067   
                                    invited             456        456   
Soft Matter                         attendee         

In [ ]:
#first step is to find a way to look at sponsoring units
#some way where all is assigne